### ספריות

In [14]:
import os
import sys
import pandas as pd

In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [16]:
cwd = os.getcwd()

create_forecast_basic_folder_path = os.path.dirname(cwd)

sys.path.append(create_forecast_basic_folder_path)

### פונקציות גלובליות

In [17]:
from global_functions import up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_forecast_version_folder_location

In [18]:
forecast_version_folder_location=get_forecast_version_folder_location(r'{}\background_files\forecast_version_folder_location.txt'.format(create_forecast_basic_folder_path))

forecast_version_folder_location=r'{}\BASE_YEAR'.format(forecast_version_folder_location)

ממשיך את הקוד עכשיו שיכלול גם את המאפיינים הגיאוגרפים שעיסא צריך

In [19]:
col=['Taz_num','geometry']

In [20]:
TAZ_V4_date = get_newest_date_file(create_forecast_basic_folder_path, 'with_geo_info')

taz=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_folder_path, TAZ_V4_date))

In [21]:
#העלה של נתוני אנשים התפלגות גילים שנוצר לפני הקוד הזה
path=r'{}\Intermediates'.format(cwd)
taz=up_load_df(path,'taz_before_add_geo')

In [22]:
taz['tazSector']=1 #ערבי
taz.loc[taz['main_secto']=='U_Orthodox','tazSector']=2
taz.loc[taz['main_secto']=='Jewish','tazSector']=3
taz.loc[taz['main_secto']=='Palestinian','tazSector']=4

לייצר פומה

In [23]:
poly_pumas=up_load_shp(r'{}\background_files\poly_pumas.shp'.format(create_forecast_basic_folder_path))
col_old=['poly_puma',  'F3', 'F2', 'F1', 'geometry']
col_new=['poly_puma',  '3', '2', '1', 'geometry']
poly_pumas=drop_geo(change_cols_names(poly_pumas,col_old,col_new))
pumas_by_poly_sector=poly_pumas.melt(id_vars='poly_puma',var_name='tazSector',value_name='PUMA')
pumas_by_poly_sector['tazSector']=pumas_by_poly_sector['tazSector'].astype(int)

In [24]:
taz=taz.merge(pumas_by_poly_sector,on=['poly_puma','tazSector'],how='left')

In [25]:
taz.loc[taz['PUMA']==0,'PUMA']=999
taz.loc[taz['pop']==0,'PUMA']=999
taz.loc[taz['main_secto']=='Palestinian','PUMA']=999
taz.loc[taz['jeru_metro']==0,'PUMA']=999

פלט של ההיברדי

In [26]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [27]:

col_needed=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slope',
'Urban']

col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

In [28]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [29]:
df=change_cols_names(taz, col_needed, col_new_name)

In [30]:
col_to_int=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

In [31]:
for c in col_to_int:
    df.loc[:, c] = df.loc[:, c].astype(int)

In [32]:
df.sort_values(by='TAZ',ascending=True).to_excel(r'{}\BaseProjections2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

DISTRICT

In [33]:
taz['DISTRICT']=999
taz.loc[(taz['jew']==0)&(taz['main_secto']!='Palestinian'),'DISTRICT']=1
taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']=='yes'),'DISTRICT']=2
taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']=='yes'),'DISTRICT']=3
taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']=='no')&(taz['jeru_metro']==1),'DISTRICT']=5
taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']=='no')&(taz['jeru_metro']==1),'DISTRICT']=6
taz.loc[taz['pop']==0,'DISTRICT']=999

In [34]:
taz.query('Taz_num==6')

,Taz_num,Taz_name,jeru_metro,in_jerusal,main_secto,SCHN_NAME,ENG_NAME_n,zonetype,Muni_Heb,Sug_Muni,CR_PNIM,highBusine,SCHOOLDIST,Urban,EIProp,poly_puma,F3,F2,F1,agri,Indus,Com_hotel,Business,Public,commuting,ieold,IEProp,perScaled,FreeBuffer,PaidBuffer,Rest_EmpBu,searchtime,walktime,cost,Agg_taz_nu,area,CITYCODE1,CITYCODE2,CITYCODE3,CITYCODE4,codeseq,codeseqCon,county,majunivenr,parktot,superZone,Taz1,slope,REGION,yosh,jerusalem_,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,TOA1,TOA2,TOA3,sector_for_chinuc,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,UNIVENRORTHFEMALE,UNIVENRORTHMALE,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,check_emp_sum,hh,tazSector,PUMA,DISTRICT
110,6,המעלות,1,yes,Jewish,מער,Jerusalem,Jerusalem,ירושלים,עירייה,3000,1,5,2,0.050,1,1,6,18,0.000,0.000,0.000,0.000,0.000,0.050,0.050,0.050,0.950,1010,568.000,91.000,6,5,4.550,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,"POLYGON Z ((220438.0235000001 631939.6228 0, 2...",0,0,0,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0.000,0.000,0.000,Jewish,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0,0.000,0.000,1,0.000,0.000,217.510,783.035,45.124,14.599,30.525,150.635,126.745,71.004,48.442,33.843,22.562,21.235,14.599,25.216,23.889,28.534,41.806,84.276,3.600,0,783.035,150.635,126.745,0,30.525,366.301,181.160,154.616,392.095,384.253,19.213,27065.578,0,0.000,0.000,0.000,0.000,217.510,3,1.000,3


In [21]:

col_needed=['Taz_num','Agg_taz_nu','PUMA','DISTRICT','REGION','SCHOOLDIST']

col_new_name=['TAZ','AGG_TAZ','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

In [22]:
change_cols_names(taz, col_needed, col_new_name).sort_values(by='TAZ',ascending=True).to_excel(r'{}\puma2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

SED

In [23]:
col_needed=['Taz_num',
 'Taz_num',
 'hh',
 'PUMA',
 'DISTRICT',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Business',
 'Public',
 'emp_Education',
 'agri',
 'total_emp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'Univ_AR',
 'Univ_SE',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCon',
 'PaidBuffer',
 'Rest_EmpBu',
 'FreeBuffer',
 'SCHOOLDIST',
 'SCHOOLDIST',
 'highBusine',
 'searchtime',
 'walktime',
 'cost']

In [24]:
col_new_name=['maz',
 'taz',
 'hh_total',
 'puma',
 'district',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Off_Bsness',
 'Public',
 'Education',
 'Agri',
 'totemp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'UNIVENRARAB',
 'UNIVENRSEC',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'PaidBuffer',
 'Rest_EmpBuffer',
 'FreeBuffer',
 'schDistrict',
 'schDistrictAgg',
 'highBusinessFlag',
 'searchtime',
 'walktime',
 'cost']

In [25]:
change_cols_names(taz, col_needed, col_new_name).sort_values(by='maz',ascending=True).to_excel(r'{}\SED_2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

In [26]:
taz.to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)